### Read the data from the input Excel file

In [30]:
import pandas as pd
inputFileName = "MineScheduling_smallDataSet.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=0)
jobDF = pd.read_excel(inputFileName, "Job", skiprows=0)
jobMachineDF = pd.read_excel(inputFileName, "JobMachine", skiprows=0)
switchingDF = pd.read_excel(inputFileName, "Switching", skiprows=0)

### Set the data into dict + compute maxDuration

In [31]:
horizonDuration = paramDF['Duration (day)'][0]

duration = dict()  #duration[vJob][vMachine]
maxDuration = 0.0
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in duration : duration[vJob] = dict() 
    duration[vJob][vMachine] = jobMachineDF['Duration'][iJob]
    if duration[vJob][vMachine] > maxDuration : maxDuration = duration[vJob][vMachine]


### Create your linear program

In [32]:
from pulp import *
prob = LpProblem("MineSchedulingProblem", LpMaximize)

### Add variables

In [33]:
#allocationVar[vJob][vMachine] & isFirst[vJob][vMachine]
allocationVar = dict()
isFirstVar = dict()  #Jobs that each machine can do
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in allocationVar : allocationVar[vJob] = dict() 
    if not vJob in isFirstVar : isFirstVar[vJob] = dict() 
    allocationVar[vJob][vMachine] = LpVariable("Allocation[%s][%s]"%(vJob,vMachine), cat='Binary')
    isFirstVar[vJob][vMachine] = LpVariable("isFirst[%s][%s]"%(vMachine, vJob), cat='Binary') 

print(isFirstVar)
print(allocationVar)
#startVar[vJob] and isOver[vJob]
startVar = dict()
isOverVar = dict() # 
for vJob in jobDF['Id'] :
    startVar[vJob] = LpVariable("Start[%s]"%(vJob), lowBound=0, upBound=horizonDuration, cat='Continuous')
    isOverVar[vJob] = LpVariable("IsOver[%s]"%(vJob), lowBound=0, cat='Binary')

    
#isSuccessor[vMachine][vJob1-vJob2] = 1 if vJob2 is the direct successor of vJob1
isSuccessorVar = dict()  # successor job after the previous job is done regarding a machine 
for iMachine,vMachine in enumerate(switchingDF['Machine']):
    vJob1 = switchingDF['Job1'][iMachine]
    vJob2 = switchingDF['Job2'][iMachine]
    if not vMachine in isSuccessorVar : isSuccessorVar[vMachine] = dict()
    if not vJob1 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob1] = dict() 
    if not vJob2 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob2] = dict() 
    isSuccessorVar[vMachine][vJob1][vJob2] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob1, vJob2), cat='Binary')



{'Job1': {'Bull': isFirst_Bull__Job1_}, 'Job2': {'Bull': isFirst_Bull__Job2_}, 'Job3': {'Bull': isFirst_Bull__Job3_, 'SmallDragline': isFirst_SmallDragline__Job3_}, 'Job4': {'SmallDragline': isFirst_SmallDragline__Job4_, 'BigDragline': isFirst_BigDragline__Job4_}, 'Job5': {'SmallDragline': isFirst_SmallDragline__Job5_, 'BigDragline': isFirst_BigDragline__Job5_}, 'Job6': {'BigDragline': isFirst_BigDragline__Job6_}, 'Job7': {'Bull': isFirst_Bull__Job7_, 'SmallDragline': isFirst_SmallDragline__Job7_}, 'Job8': {'SmallDragline': isFirst_SmallDragline__Job8_, 'BigDragline': isFirst_BigDragline__Job8_}, 'Job9': {'SmallDragline': isFirst_SmallDragline__Job9_, 'BigDragline': isFirst_BigDragline__Job9_}, 'Job10': {'BigDragline': isFirst_BigDragline__Job10_}, 'Job11': {'Bull': isFirst_Bull__Job11_}, 'Job12': {'Bull': isFirst_Bull__Job12_}, 'Job13': {'Bull': isFirst_Bull__Job13_, 'SmallDragline': isFirst_SmallDragline__Job13_}, 'Job14': {'SmallDragline': isFirst_SmallDragline__Job14_, 'BigDragline

### Add objective

In [34]:
Production=jobDF.set_index('Id').to_dict()['Production']  # Job & Production


obj = LpAffineExpression()
obj += lpSum([isOverVar[j] * Production[j] for j in jobDF['Id']])
prob += obj


### Add constraints

In [35]:
#1Max one allocation per job
#Define a dictionary : key = Job, value = Associated machines
for vJob in jobDF['Id']:
    prob += lpSum([allocationVar[vJob][vMachine] for vMachine in allocationVar[vJob].keys()])<=1


In [36]:
#2Max one "isFirst" job per
for vMachine in paramDF['Machine']:
        vJobs=jobMachineDF.loc[jobMachineDF['Machine'] == vMachine , 'Job']
        prob += lpSum([isFirstVar[vJob][vMachine] for vJob in vJobs])<=1, "IsFirstOnMachine[%s]"%(vMachine)

In [37]:
#3Constraint stating the "start" variable
#Start[j’] >= Start[j] + duration[j][m] + switchingTime[m][j][j’]  - BigM * (1 - isSuccessor[m][j][j’])
#Start[j'] - Start[j] - BigM * isSuccessor[m][j][j'] >= duration[j][m] + switchingTime[m][j][j’] - BigM
BigM=100000
for iMachine,vMachine in enumerate(switchingDF['Machine']):
    Job1=str(switchingDF['Job1'][iMachine])
    Job2=str(switchingDF['Job2'][iMachine])
    duration = float(jobMachineDF.loc[(jobMachineDF['Machine']==vMachine)&(jobMachineDF['Job']==Job1),'Duration'])
    duration2 = float(jobMachineDF.loc[(jobMachineDF['Machine']==vMachine)&(jobMachineDF['Job']==Job2),'Duration'])
    SwitchingTime = float(switchingDF['SwitchingTime (day)'][iMachine])
    #print(Job1)
    #print(Job2)
    #print(duration)
    #print(SwitchingTime)
    prob+=startVar[Job2]>=startVar[Job1] + duration + SwitchingTime-BigM*(1-isSuccessorVar[vMachine][Job1][Job2])

    

In [38]:
#4Constraint stating the "isOver" variable
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] <= durationHorizon + BigM * (1 - isOver[j])
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] + BigM * isOver[j] <= durationHorizon + BigM

for vJob in jobDF['Id']:
    duration = jobMachineDF[jobMachineDF['Job']==vJob]
    durationdict=dict(zip(duration.Machine, duration.Duration))
    prob+=startVar[vJob]+lpSum([durationdict[vMachine]*allocationVar[vJob][vMachine] for vMachine in durationdict.keys()])<=horizonDuration+BigM*(1-isOverVar[vJob])
    

In [39]:
#Define a dictionary : key = Job, value = Associated machines
MachineJob_dict = dict(jobMachineDF.groupby('Job')['Machine'].apply(list))

In [40]:
#5Constraint stating that if isOver = 1, then at least one allocation = 1
for vJob in jobDF['Id']:
    prob+=  isOverVar[vJob] <= lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob] )
    

In [41]:
#6 If a Job happens, it is either a successor or a first to be done :
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if vMachine in isSuccessorVar:
        prob += allocationVar[vJob][vMachine] <= isFirstVar[vJob][vMachine] + lpSum(isSuccessorVar[vMachine][vJob2][vJob] for vJob2 in switchingDF.loc[(switchingDF["Machine"]==vMachine)&(switchingDF["Job2"]==vJob),"Job1"])
        

In [42]:
#7 Max one successor per job
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if vMachine in isSuccessorVar:
        prob += lpSum(isSuccessorVar[vMachine][vJob][OtherJob] for OtherJob in isSuccessorVar[vMachine][vJob]) <= 1
        

In [43]:

machine = paramDF[['Machine']]
machine_names = machine.Machine.unique()
Jobmachine_dict=dict(jobMachineDF.groupby('Machine')['Job'].apply(list))   

In [44]:
#8 Constraint stating that IsSuccessor[m][j1][j2] = 1 --> allocation[j1][m] = 1

for i in range(len(switchingDF['Machine'])):
    vMachine=switchingDF['Machine'][i]
    Job1=switchingDF['Job1'][i]
    Job2=switchingDF['Job2'][i]
    prob += isSuccessorVar[vMachine][Job1][Job2] <= allocationVar[Job1][vMachine]


In [45]:
#9 Constraint stating that if start > 0, then then at least one allocation = 1
for vJob in jobDF['Id'] :
    prob += startVar[vJob] <= horizonDuration * lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob] )
    
    

In [46]:
#10 If Sum allocation = 1, then start >= 1.0
for vJob in jobDF['Id']:
    prob +=  lpSum(allocationVar[vJob][vMachine] for vMachine in MachineJob_dict[vJob]) <= startVar[vJob]
    

In [47]:
##11 Constraint imposing predecessors : isOver[j] = 1  isOver[vPred] = 1  
jPred_dict=dict(zip(jobDF['Id'], jobDF['PredecessorId'])) 
for j in jobDF['Id']:
    if jPred_dict[j] in jobDF['Id']:
        prob +=  isOverVar[j] <= isOverVar[jPred_dict[j]]


In [48]:
#12 Constraint imposing predecessors :   j shall start only once vPred has finished

horizonDuration = paramDF['Duration (day)'][0]

duration = dict()  #duration[vJob][vMachine]
maxDuration = 0.0
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in duration : duration[vJob] = dict() 
    duration[vJob][vMachine] = jobMachineDF['Duration'][iJob]
    if duration[vJob][vMachine] > maxDuration : maxDuration = duration[vJob][vMachine]

for j in jobDF['Id']:
    jPred=jPred_dict[j]
    if jPred not in jobDF['Id']:
        continue
    prob += StartVar[jPred] + lpSum([duration[jPred][m] * allocationVar[jPred][m] for m in MachineJ_dict[jPred]])<= StartVar[j]+BigM*(1-lpSum([allocationVar[j][m] for m in MachineJ_dict[jPred]])),"Predecessor_Start[%s]_[%s]"%(j,jPred)
    
    

### Solve and display results

In [49]:
prob.writeLP("MineSchedulingProblem", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
varsDict = {}
for v in prob.variables():
    varsDict[v.name] = v.varValue
    if "IsOver" in v.name or "Start" in v.name or "Allocation" in v.name or "IsSuccessor" in v.name:
        if v.varValue != 0.0 : print(v.name, "=", v.varValue)

Status: Optimal
Objective =  489.0
Allocation_Job10__BigDragline_ = 1.0
Allocation_Job11__Bull_ = 1.0
Allocation_Job12__Bull_ = 1.0
Allocation_Job13__SmallDragline_ = 1.0
Allocation_Job14__BigDragline_ = 1.0
Allocation_Job18__BigDragline_ = 1.0
Allocation_Job1__Bull_ = 1.0
Allocation_Job3__SmallDragline_ = 1.0
IsOver_Job10_ = 1.0
IsOver_Job11_ = 1.0
IsOver_Job12_ = 1.0
IsOver_Job13_ = 1.0
IsOver_Job14_ = 1.0
IsOver_Job18_ = 1.0
IsOver_Job1_ = 1.0
IsOver_Job3_ = 1.0
IsSuccessor_BigDragline__Job10__Job14_ = 1.0
IsSuccessor_BigDragline__Job14__Job18_ = 1.0
IsSuccessor_Bull__Job11__Job12_ = 1.0
IsSuccessor_Bull__Job1__Job11_ = 1.0
IsSuccessor_SmallDragline__Job3__Job13_ = 1.0
Start_Job10_ = 1.0
Start_Job11_ = 22.2
Start_Job12_ = 72.0
Start_Job13_ = 56.615385
Start_Job14_ = 27.0
Start_Job18_ = 58.461538
Start_Job1_ = 1.0
Start_Job3_ = 1.0


In [50]:
!pip install python-gantt

In [51]:
!pip install datetime

In [52]:
duration

{'Job1': {'Bull': 17.2},
 'Job2': {'Bull': 28.266666666666666},
 'Job3': {'Bull': 54.0, 'SmallDragline': 45.0},
 'Job4': {'SmallDragline': 46.90909090909091,
  'BigDragline': 36.857142857142854},
 'Job5': {'SmallDragline': 28.8, 'BigDragline': 24.0},
 'Job6': {'BigDragline': 54.0},
 'Job7': {'Bull': 34.8, 'SmallDragline': 29.0},
 'Job8': {'SmallDragline': 30.90909090909091,
  'BigDragline': 24.285714285714285},
 'Job9': {'SmallDragline': 34.4, 'BigDragline': 28.666666666666668},
 'Job10': {'BigDragline': 24.0},
 'Job11': {'Bull': 26.095238095238095},
 'Job12': {'Bull': 28.0},
 'Job13': {'Bull': 51.27272727272727, 'SmallDragline': 43.38461538461539},
 'Job14': {'SmallDragline': 26.333333333333332,
  'BigDragline': 21.066666666666666},
 'Job15': {'SmallDragline': 18.90909090909091, 'BigDragline': 16.0},
 'Job16': {'BigDragline': 22.545454545454547},
 'Job17': {'SmallDragline': 47.333333333333336,
  'BigDragline': 37.86666666666667},
 'Job18': {'SmallDragline': 49.09090909090909,
  'BigDr

In [53]:
print(isFirstVar["Job1"]['Bull'].varValue)

1.0


In [55]:
import datetime
import gantt
duration = dict()  #duration[vJob][vMachine]
maxDuration = 0.0
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in duration : duration[vJob] = dict() 
    duration[vJob][vMachine] = jobMachineDF['Duration'][iJob]
    if duration[vJob][vMachine] > maxDuration : maxDuration = duration[vJob][vMachine]

durationdict=duration

# Change font default
gantt.define_font_attributes(fill='black',
                             stroke='black',
                             stroke_width=0,
                             font_family="Verdana")


# Création des variables ressources
my_vars={}
for iMac in paramDF['Machine']:
    my_vars[iMac]=gantt.Resource(iMac)
       
# Create a project
p = gantt.Project(name='Mining Scheduling Problem')

task_vars={}
def successorship(FirstJob):
    global task_vars
    global p
    global isSuccessorVar
    global vMachine
    for oJob in isSuccessorVar[vMachine][FirstJob].keys():
        if isSuccessorVar[vMachine][FirstJob][oJob].varValue==1:
            #Add Switching :
            SwitchingTask="Switching Time"
            try:
                SwitchingDuration=float(switchingDF.loc[(switchingDF['Job1']==FirstJob) & (switchingDF['Job2']==oJob) & (switchingDF['Machine']==vMachine) , 'SwitchingTime (day)'])
            except:
                SwitchingDuration=float(switchingDF.loc[(switchingDF['Job1']==oJob) & (switchingDF['Job2']==FirstJob) & (switchingDF['Machine']==vMachine) , 'SwitchingTime (day)'])
            ### herenswitching
            print(SwitchingDuration)
            print(vMachine,FirstJob,oJob)
            #Add to the Task :
            Task=oJob
            duration=jobMachineDF.loc[(jobMachineDF['Job']==oJob) & (jobMachineDF['Machine']==vMachine),'Duration']
            ###add here next task
            return successorship(Task)
    
for vMachine in paramDF['Machine']:
    JobsD=jobMachineDF[jobMachineDF['Machine']==vMachine]['Job']
    FirstJob=""
    for vJob in JobsD :
        if isFirstVar[vJob][vMachine].varValue==1 :
            FirstJob=vJob

    print(FirstJob)
    
    FirstJobDuration=float(jobMachineDF.loc[(jobMachineDF['Job']==FirstJob) & (jobMachineDF['Machine']==vMachine) , 'Duration'])
    
    print(FirstJobDuration)
    
    task_vars[FirstJob] = gantt.(name=FirstJob,start=datetime.date(2021, 1, 1) + datetime.timedelta(days=startVar[FirstJob].varValue),duration=durationdict[FirstJob][vMachine],resources=[vMachine])
    
    successorship(FirstJob)
    continue
    ##Code to add the task to gantt
    
    
#for n in allocationVar.keys():
#    if allocationVar[n]!= 0: 
#        print(n[0],n[1],startVar[n[0]], jobMachineDF[n[0],n[1]])
#
#        task_vars[n[0]] = gantt.Task(name=n[0],
#                start=datetime.date(2021, 1, 1) + datetime.timedelta(days=startVar[n[0]].varValue),     
#                duration=durationdict[n[0],n[1]],
#                resources=[my_vars[n[1]]])
        
#        p.add_task(task_vars[n[0]])

############################$ MAKE DRAW ###############
#p.make_svg_for_tasks(filename='MSGantt.svg',
#                      today=datetime.date(2021, 1, 1))

##########################$ /MAKE DRAW ###############

SyntaxError: invalid syntax (<ipython-input-55-9af56c0a937a>, line 64)

In [ ]:
key(isSuccessorVar["Bull"]["Job1"]["Job2"])